# APUNTES

In [20]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array


print("TensorFlow versión:", tf.__version__)
print("Keras versión:", tf.keras.__version__)

# Intentar importar los módulos
from tensorflow.keras.preprocessing import image
print("Módulo keras.preprocessing importado correctamente.")

TensorFlow versión: 2.19.0
Keras versión: 3.9.1
Módulo keras.preprocessing importado correctamente.


In [21]:
def prepare_data(dogs_path, cats_path, val_size=0.15, test_size=0.15, random_state=42):
    # Obtener listas de archivos en cada carpeta
    files_dogs = os.listdir(dogs_path)
    files_cats = os.listdir(cats_path)

    # Crear DataFrame con categorías
    df_dogs = pd.DataFrame({'filename': files_dogs, 'category': 1})  # 1 para perros
    df_cats = pd.DataFrame({'filename': files_cats, 'category': 0})  # 0 para gatos

    # Agregar prefijos para indicar las rutas completas
    df_dogs["filename"] = df_dogs["filename"].apply(lambda x: os.path.join(dogs_path, x))
    df_cats["filename"] = df_cats["filename"].apply(lambda x: os.path.join(cats_path, x))

    # Unir ambos DataFrames
    df = pd.concat([df_dogs, df_cats], ignore_index=True)

    # Dividir en entrenamiento, validación y prueba
    train_df, temp_df = train_test_split(df, test_size=(val_size + test_size), stratify=df["category"], random_state=random_state)
    val_ratio = val_size / (val_size + test_size)  
    val_df, test_df = train_test_split(temp_df, test_size=(1 - val_ratio), stratify=temp_df["category"], random_state=random_state)

    return train_df, val_df, test_df

# Definir rutas de datos
current_dir = os.getcwd()
files_path_train = os.path.join(current_dir, "../data/raw/train")

dogs_path = os.path.join(files_path_train, "dogs")
cats_path = os.path.join(files_path_train, "cats")

# Llamar a la función
train_df, val_df, test_df = prepare_data(dogs_path, cats_path)

# Mostrar algunas filas para verificar
print(train_df.head())


                                                filename  category
11893  c:\Users\mamen\OneDrive\Documentos\GitHub\deep...         1
23384  c:\Users\mamen\OneDrive\Documentos\GitHub\deep...         0
3919   c:\Users\mamen\OneDrive\Documentos\GitHub\deep...         1
19096  c:\Users\mamen\OneDrive\Documentos\GitHub\deep...         0
218    c:\Users\mamen\OneDrive\Documentos\GitHub\deep...         1


In [22]:
print(f"Total Training Images: {len(train_df)}")
print(f"Total Validation Images: {len(val_df)}")
print(f"Total Test Images: {len(test_df)}")

Total Training Images: 17500
Total Validation Images: 3750
Total Test Images: 3750


In [23]:

# def load_and_process_images(dataframe, img_size=(200, 200)):
#     X = []  # Lista para imágenes
#     y = []  # Lista para etiquetas
    
#     for index, row in dataframe.iterrows():
#         img_path = row["filename"]  # Ruta completa de la imagen
#         label = row["category"]  # Etiqueta (1 = perro, 0 = gato)
        
#         # Cargar imagen y redimensionar
#         img = load_img(img_path, target_size=img_size)  
#         img_array = img_to_array(img) / 255.0  # Normalizar valores entre 0 y 1
        
#         # Guardar en listas
#         X.append(img_array)
#         y.append(label)

#     # Convertir listas a arrays de NumPy
#     X = np.array(X, dtype=np.float32)
#     y = np.array(y, dtype=np.int32)

#     return X, y

# # Cargar imágenes del conjunto de entrenamiento
# X_train, y_train = load_and_process_images(train_df)

# # Mostrar forma de los datos
# print("Tamaño de X_train:", X_train.shape)  # Esperado: (25000, 200, 200, 3)
# print("Tamaño de y_train:", y_train.shape)  # Esperado: (25000,)


In [24]:
# processed_path = os.path.join(current_dir, "../data/processed")
# save_train_path = os.path.join(processed_path, "train_data.npz")
# np.savez(save_train_path, X_train=X_train, y_train=y_train)

In [25]:
# Si tengo guardado en el archivo, lo extraigo de save_train_path
# # 📌 Cargar el archivo .npz
# data = np.load("train_data.npz")

# # 📌 Asignar los datos a variables
# X_train = data["X_train"]
# y_train = data["y_train"]

# # 📌 Verificar dimensiones
# print("X_train shape:", X_train.shape)
# print("y_train shape:", y_train.shape)

In [26]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directorios de imágenes
train_dir = os.path.join(current_dir, "../data/raw/train")
test_dir = os.path.join(current_dir, "../data/raw/test1/unknown")

# 📌 Crear objeto ImageDataGenerator para AUMENTACIÓN y normalización
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normaliza valores entre 0-1
    rotation_range=20,   # Rotación aleatoria
    width_shift_range=0.2,  # Desplazamiento horizontal
    height_shift_range=0.2, # Desplazamiento vertical
    shear_range=0.2,  # Transformación de corte
    zoom_range=0.2,   # Zoom aleatorio
    horizontal_flip=True,  # Volteo horizontal
    fill_mode="nearest",  # Rellenar huecos con valores cercanos
    validation_split=0.2  # 🔹 20% de los datos serán de validación
)

# 📌 Crear lista de archivos
test_files = os.listdir(test_dir)

# 📌 Crear un DataFrame (sin etiquetas, ya que son desconocidas)
test_df = pd.DataFrame({'filename': test_files})

# 📌 Crear un ImageDataGenerator (solo normalización)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# 📌 Cargar imágenes de entrenamiento con etiquetas AUTOMÁTICAS
train_generator = train_datagen.flow_from_directory(
    train_dir,  
    target_size=(200, 200),  # Redimensiona las imágenes
    batch_size=32,  # Número de imágenes por lote
    class_mode="binary",  # Etiquetas binarias (0 = gato, 1 = perro)
    subset="training"  # 🔹 Usa 80% para entrenamiento
)
# 📌 Generador de validación
val_generator = train_datagen.flow_from_directory(
    train_dir,  
    target_size=(200, 200),
    batch_size=32,
    class_mode="binary",
    subset="validation"  # 🔹 Usa 20% para validación
)

# 📌 Cargar imágenes del test (sin etiquetas)
test_generator = test_datagen.flow_from_dataframe(
    test_df,  # DataFrame con los archivos
    directory=test_dir,  # Ruta donde están las imágenes
    x_col="filename",  # Nombre de la columna con los archivos
    target_size=(224, 224),  # Redimensionar imágenes
    batch_size=32,
    class_mode=None,  # ⚠️ No hay etiquetas, solo cargamos imágenes
    shuffle=False  # Mantener el orden original
)

# 📌 Mostrar clases detectadas
print("Clases detectadas:", train_generator.class_indices)


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Found 12500 validated image filenames.
Clases detectadas: {'cats': 0, 'dogs': 1}


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# 🔹 1. Definir el modelo basado en la arquitectura dada
model = Sequential()
model.add(Conv2D(input_shape=(200, 200, 3), filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096, activation="relu"))
model.add(Dense(units=4096, activation="relu"))
model.add(Dense(units=1, activation="sigmoid"))  # 2 clases: perros y gatos. 1 neurona sigmoid

# 🔹 2. Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.0001),  # Optimizador Adam con tasa de aprendizaje baja
              loss="categorical_crossentropy",  # Pérdida para clasificación multiclase
              metrics=["accuracy"])  # Medir precisión

# 🔹 3. Definir un Early Stopping para evitar sobreajuste
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# 🔹 4. Entrenar el modelo
history = model.fit(
    train_generator,  # Datos de entrenamiento
    epochs=5,  # Número de épocas recomendable 20
    batch_size=32,  
    callbacks=[early_stop]
)

# 🔹 5. Evaluar rendimiento en datos de prueba
test_loss, test_acc = model.evaluate(test_generator)
print(f"\n🔹 Precisión en prueba: {test_acc:.4f}")


Epoch 1/5


c:\Users\mamen\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\losses\losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


102/625 ━━━━━━━━━━━━━━━━━━━━ 4:30:42 31s/step - accuracy: 0.5014 - loss: 0.0000e+00

In [ ]:
# 📌 Hacer predicciones
predictions = model.predict(test_generator)

# 📌 Convertir predicciones a etiquetas (0 = gato, 1 = perro)
predicted_labels = np.argmax(predictions, axis=1)

# 📌 Agregar etiquetas al DataFrame
test_df["label"] = predicted_labels
test_df["label"] = test_df["label"].map({0: "cat", 1: "dog"})  # Convertir 0/1 a texto

# 📌 Mostrar los primeros resultados
print(test_df.head())

# 📌 Guardar resultados en CSV
test_df.to_csv("test_predictions.csv", index=False)